In [298]:
from polars import DataFrame, col,lit
from polars import Float64, Int64

In [299]:
from collections import namedtuple
Tx = namedtuple("Tx", ("ts", "price", "qty"))

txs = [
    Tx(1, 102, 2),
    Tx(3, 105, 5),
    Tx(4, 105.5, -6),
    Tx(5, 106.5, 3),
    Tx(7, 108, 1),
]

In [300]:


def condsolidate_txs(
    txs: list,
    *,
    period_start_ts:float,
    period_start_price:float,
) -> DataFrame:
    TXs = DataFrame(txs)
    before = TXs.filter(col("ts") <= period_start_ts)
    after = TXs.filter(col("ts") > period_start_ts)

    before = before.select(
        # col("ts").replace_strict({}, default=period_start_ts).cast(Int64),
        ts=lit(period_start_ts).cast(Int64),
        # col("price").replace_strict({}, default=period_start_price).cast(Float64),
        price=lit(period_start_price).cast(Float64),
        qty=col("qty").sum(),
    ).tail(1)

    return before.vstack(after)


def calc_returns(
    txs: list, *, period_start_ts:int, period_start_price:float, market_price:float,
) -> dict:
    TXs = condsolidate_txs(
        txs,
        period_start_ts=period_start_ts,
        period_start_price=period_start_price,
    )
    TXs = TXs.with_columns(
        ts_start=col("ts"),
        ts_end=col("ts").shift(
            -1
        ),  # fill_null with current_timestamp or ts of market_price
        price_start=col("price"),
        price_end=col("price").shift(-1).fill_null(market_price),
        running_holding=col("qty").cum_sum(),
        cost=col("qty") * col("price"),
    ).with_columns(
        holding_before=col("running_holding").shift(1).fill_null(0),
        growth_factor=col("price_end") / col("price_start"),
        running_value=col("running_holding") * col("price_end"),
        running_cost=col("cost").cum_sum(),
    )

    returns = TXs.select(
        twr=col("growth_factor").product() - 1,
        final_value=col("running_value").last(),
        final_cost=col("cost").sum(),
        start_value=(col("holding_before")*col("price_start")).first()
    ).with_columns(
        dollar_return=col('final_value') - col('start_value') - col('final_cost')
    ).with_columns(
        mwr=col('dollar_return') / col('final_cost')
    )

    return returns.to_dicts()[0]



In [301]:
market_price = 110
period_start_ts = 0
period_start_price = 101

res = calc_returns(
    txs,
    period_start_price=period_start_price,
    period_start_ts=period_start_ts,
    market_price=market_price,
)

twr, final_value, final_cost, start_value, dollar_return, mwr = res.values()


print(f"""
    {twr=}
    {mwr=}
    {start_value=}
    {final_value=}
    {final_cost=}
    {dollar_return=}
""")


    twr=0.08910891089108897
    mwr=0.050620821394460364
    start_value=0.0
    final_value=550.0
    final_cost=523.5
    dollar_return=26.5

